#### linear, ReLU, sigmoid, tanh, and softmax activation functions as a class for neural networks implementation

In [1]:
import numpy as np

class LinearActivation:
    def forward(self, x):
        return x

class ReLUActivation:
    def forward(self, x):
        return np.maximum(0, x)

class SigmoidActivation:
    def forward(self, x):
        return 1 / (1 + np.exp(-x))

class TanhActivation:
    def forward(self, x):
        return np.tanh(x)

class SoftmaxActivation:
    def forward(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

# Example usage
x = np.array([-1, 0, 1])

relu = ReLUActivation()
print("ReLU:", relu.forward(x))

sigmoid = SigmoidActivation()
print("Sigmoid:", sigmoid.forward(x))

tanh = TanhActivation()
print("Tanh:", tanh.forward(x))

softmax = SoftmaxActivation()
print("Softmax:", softmax.forward(np.array([x, x+1])))


ReLU: [0 0 1]
Sigmoid: [0.26894142 0.5        0.73105858]
Tanh: [-0.76159416  0.          0.76159416]
Softmax: [[0.26894142 0.26894142 0.26894142]
 [0.73105858 0.73105858 0.73105858]]


#### class structure and forward propagation including the loss (cost) function implementation for a deep (multilayer) neural network

In [3]:
import numpy as np

# Activation Function Classes (as previously defined)
# ... (ReLUActivation, SigmoidActivation, etc.)

class Layer:
    def __init__(self, input_size, output_size, activation_function=None):
        self.weights = np.random.randn(output_size, input_size) * 0.01
        self.biases = np.zeros((output_size, 1))
        self.activation_function = activation_function
        self.output = None
        self.input = None

    def forward(self, input_data):
        self.input = input_data
        self.output = np.dot(self.weights, self.input) + self.biases
        if self.activation_function:
            self.output = self.activation_function.forward(self.output)
        return self.output

class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward_propagation(self, input_data):
        output = input_data
        for layer in self.layers:
            output = layer.forward(output)
        return output

    def compute_loss(self, predicted_output, actual_output):
        return np.mean(np.power(predicted_output - actual_output, 2))

# Example Usage
nn = NeuralNetwork()
nn.add_layer(Layer(3, 5, ReLUActivation()))
nn.add_layer(Layer(5, 2, SigmoidActivation()))

# Dummy Data
X = np.random.randn(3, 1)  # Input
Y = np.array([[0], [1]])  # Actual Output

# Forward Propagation
predicted_output = nn.forward_propagation(X)

# Compute Loss
loss = nn.compute_loss(predicted_output, Y)
print("Loss:", loss)


Loss: 0.2500565296786265


#### Backpropagation implementation for a deep (multilayer) neural network

In [4]:
import numpy as np

# Activation Function Classes
class ReLUActivation:
    def forward(self, x):
        return np.maximum(0, x)
    
    def backward(self, output_error, output):
        # ReLU derivative is 0 for output < 0 and 1 for output >= 0
        return output_error * (output > 0)

class SigmoidActivation:
    def forward(self, x):
        return 1 / (1 + np.exp(-x))
    
    def backward(self, output_error, output):
        # Sigmoid derivative
        return output_error * (output * (1 - output))

class Layer:
    def __init__(self, input_size, output_size, activation_function=None):
        self.weights = np.random.randn(output_size, input_size) * 0.01
        self.biases = np.zeros((output_size, 1))
        self.activation_function = activation_function
        self.output = None
        self.input = None
        self.dweights = None
        self.dbiases = None

    def forward(self, input_data):
        z = np.dot(self.weights, input_data) + self.biases
        self.input = input_data
        self.output = self.activation_function.forward(z) if self.activation_function else z
        return self.output

    def backward(self, output_error, learning_rate):
        # Apply derivative of activation function if any
        if self.activation_function:
            output_error = self.activation_function.backward(output_error, self.output)

        self.dweights = np.dot(output_error, self.input.T)
        self.dbiases = output_error

        # Calculate error for the previous layer
        input_error = np.dot(self.weights.T, output_error)
        
        # Update parameters
        self.weights -= learning_rate * self.dweights
        self.biases -= learning_rate * self.dbiases

        return input_error

class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward_propagation(self, input_data):
        output = input_data
        for layer in self.layers:
            output = layer.forward(output)
        return output

    def compute_loss(self, predicted_output, actual_output):
        return np.mean(np.power(predicted_output - actual_output, 2))

    def compute_loss_derivative(self, predicted_output, actual_output):
        return 2 * (predicted_output - actual_output) / actual_output.size

    def backward_propagation(self, loss_derivative, learning_rate):
        gradient = loss_derivative
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient, learning_rate)

# Example Usage
nn = NeuralNetwork()
nn.add_layer(Layer(3, 5, ReLUActivation()))
nn.add_layer(Layer(5, 2, SigmoidActivation()))

# Dummy Data
X = np.random.randn(3, 1)  # Input
Y = np.array([[0], [1]])  # Actual Output

# Training loop
for i in range(1000):
    # Forward Propagation
    predicted_output = nn.forward_propagation(X)

    # Compute Loss
    loss = nn.compute_loss(predicted_output, Y)

    # Backward Propagation
    loss_derivative = nn.compute_loss_derivative(predicted_output, Y)
    nn.backward_propagation(loss_derivative, learning_rate=0.01)

    if i % 100 == 0:
        print(f'Epoch {i}, Loss: {loss}')

# Testing
predicted_output = nn.forward_propagation(X)
loss = nn.compute_loss(predicted_output, Y)
print("Final Loss:", loss)

Epoch 0, Loss: 0.24985624335147275
Epoch 100, Loss: 0.22030627207099907
Epoch 200, Loss: 0.19430862605185512
Epoch 300, Loss: 0.17138420802532356
Epoch 400, Loss: 0.15088548404440977
Epoch 500, Loss: 0.13207444428329002
Epoch 600, Loss: 0.11424647780091055
Epoch 700, Loss: 0.09693454793356139
Epoch 800, Loss: 0.08014780294811194
Epoch 900, Loss: 0.06444339274483388
Final Loss: 0.050632239400252396
